In [ ]:
!pip install ortools -q

In [48]:
from ortools.sat.python import cp_model
import time
import pandas as pd

# Parameters
num_nurses = 16
num_days = 30
num_shifts = 2  # 0 = Morning, 1 = Evening
days_per_week = 7
shifts_per_week = days_per_week * num_shifts
weekends = [6, 7, 13, 14, 20, 21, 27, 28]  # Saturdays and Sundays in 30-day month starting from Monday

# Shift requirements
morning_nurses_weekdays = 6
evening_nurses_weekdays = 4
morning_nurses_weekends = 4
evening_nurses_weekends = 4

# Constraints
max_shifts_per_month = 20
min_shifts_per_month = 17

def main():
    # Create the model
    model = cp_model.CpModel()

    # Create variables
    shifts = {}
    for n in range(num_nurses):
        for d in range(num_days):
            for s in range(num_shifts):
                shifts[(n, d, s)] = model.NewBoolVar(f'nurse{n}_day{d}_shift{s}')

    # Constraint: Each shift is assigned to the required number of nurses
    for d in range(num_days):
        for s in range(num_shifts):
            if d + 1 in weekends:
                required_nurses = morning_nurses_weekends if s == 0 else evening_nurses_weekends
            else:
                required_nurses = morning_nurses_weekdays if s == 0 else evening_nurses_weekdays
            model.Add(sum(shifts[(n, d, s)] for n in range(num_nurses)) == required_nurses)

    # Constraint: Each nurse works at most one shift per day
    for n in range(num_nurses):
        for d in range(num_days):
            model.Add(sum(shifts[(n, d, s)] for s in range(num_shifts)) <= 1)

    # # Constraint: Each nurse works between min_shifts_per_month and max_shifts_per_month shifts
    for n in range(num_nurses):
        model.Add(sum(shifts[(n, d, s)] for d in range(num_days) for s in range(num_shifts)) >= min_shifts_per_month)
        model.Add(sum(shifts[(n, d, s)] for d in range(num_days) for s in range(num_shifts)) <= max_shifts_per_month)

    # Constraint: After an evening shift, a nurse can't work a morning shift
    for n in range(num_nurses):
        for d in range(num_days - 1):
            model.Add(shifts[(n, d + 1, 0)] == 0).OnlyEnforceIf(shifts[(n, d, 1)])

    # Hard constraint: Each nurse should be free every other pair of weekends
    for n in range(num_nurses):
        for i in range(0, len(weekends)-2, 2):
            weekend1 = sum(shifts[(n, d - 1, s)] for d in weekends[i:i+2] for s in range(num_shifts))
            weekend2 = sum(shifts[(n, d - 1, s)] for d in weekends[i+2:i+4] for s in range(num_shifts))

            # Create boolean variables for each weekend pair
            works_weekend1 = model.NewBoolVar(f'nurse{n}_works_weekend{i//4}_1')
            works_weekend2 = model.NewBoolVar(f'nurse{n}_works_weekend{i//4}_2')

            # Link the boolean variables to the actual shifts
            model.Add(weekend1 > 0).OnlyEnforceIf(works_weekend1)
            model.Add(weekend1 == 0).OnlyEnforceIf(works_weekend1.Not())
            model.Add(weekend2 > 0).OnlyEnforceIf(works_weekend2)
            model.Add(weekend2 == 0).OnlyEnforceIf(works_weekend2.Not())

            # Ensure exactly one of the weekend pairs is worked
            model.Add(works_weekend1 + works_weekend2 == 1)



    # New constraints: Shift restrictions for specific nurses
    morning_only_nurse = 0  # First nurse works only morning shifts
    evening_only_nurses = [1, 2]  # Second and third nurses work only evening shifts

    # Morning-only nurse constraint
    for d in range(num_days):
        model.Add(shifts[(morning_only_nurse, d, 1)] == 0)  # Can't work evening shifts

    # Evening-only nurses constraints
    for n in evening_only_nurses:
        for d in range(num_days):
            model.Add(shifts[(n, d, 0)] == 0)  # Can't work morning shifts

    # Solve the model
    solver = cp_model.CpSolver()
    solver.parameters.max_time_in_seconds = 300.0  # Set a time limit of 5 minutes
    print("Solving...")
    start_time = time.time()
    status = solver.Solve(model)
    end_time = time.time()

    # Create and print the DataFrame
    if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
        print(f"Solution found in {end_time - start_time:.2f} seconds:")

        # Create a DataFrame to store the schedule
        schedule_data = []
        for n in range(num_nurses):
            nurse_schedule = []
            for d in range(num_days):
                if solver.Value(shifts[(n, d, 0)]) == 1:
                    nurse_schedule.append('M')  # Morning shift
                elif solver.Value(shifts[(n, d, 1)]) == 1:
                    nurse_schedule.append('E')  # Evening shift
                else:
                    nurse_schedule.append('-')  # Day off

            total_days_worked = sum(1 for day in nurse_schedule if day != '-')
            nurse_schedule.append(total_days_worked)
            schedule_data.append(nurse_schedule)

        columns = [f'Day {d+1}' for d in range(num_days)] + ['Total Days Worked']
        df = pd.DataFrame(schedule_data, columns=columns, index=[f'Nurse {n}' for n in range(num_nurses)])

        print(df)
        print("\nM: Morning shift, E: Evening shift, -: Day off")
        print("\nNote: Nurse 0 works only morning shifts, Nurses 1 and 2 work only evening shifts")

        # Save the DataFrame to a CSV file
        df.to_csv('nurse_schedule.csv')
        print("\nSchedule has been saved to 'nurse_schedule.csv'")
    elif status == cp_model.INFEASIBLE:
        print("The problem is provably infeasible.")
    elif status == cp_model.MODEL_INVALID:
        print("The model is invalid.")
    elif status == cp_model.UNKNOWN:
        print("The solver hit the time limit.")
    else:
        print("No solution found.")

    print(f"Solving process took {end_time - start_time:.2f} seconds")
    print(f"Branches explored: {solver.NumBranches()}")
    print(f"Conflicts encountered: {solver.NumConflicts()}")

if __name__ == "__main__":
    main()

Solving...
Solution found in 0.07 seconds:
         Day 1 Day 2 Day 3 Day 4 Day 5 Day 6 Day 7 Day 8 Day 9 Day 10  ...  \
Nurse 0      -     -     M     -     -     M     M     M     M      M  ...   
Nurse 1      E     -     -     E     E     E     E     E     E      E  ...   
Nurse 2      -     E     E     E     -     E     E     E     -      -  ...   
Nurse 3      M     E     -     M     M     -     -     M     M      -  ...   
Nurse 4      M     M     M     M     E     -     -     -     -      E  ...   
Nurse 5      M     -     M     M     M     -     -     M     M      M  ...   
Nurse 6      -     M     -     E     E     -     -     E     E      -  ...   
Nurse 7      -     E     -     M     M     M     M     E     -      M  ...   
Nurse 8      E     -     M     -     -     -     -     -     -      M  ...   
Nurse 9      M     M     M     -     E     -     -     M     M      -  ...   
Nurse 10     -     M     E     -     M     -     -     M     M      E  ...   
Nurse 11     E     E 

In [ ]:
from google.colab import files
files.download('nurse_schedule.csv')